# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
import torch as torch
import torch.nn.functional as F
from torch import optim as optim
import matplotlib.pyplot as plt
from collections import namedtuple,deque
import random
import copy

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="./Tennis.app")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations        # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

Score (max over agents) from episode 1: 0.0
Score (max over agents) from episode 2: 0.0
Score (max over agents) from episode 3: 0.0
Score (max over agents) from episode 4: 0.0
Score (max over agents) from episode 5: 0.09000000171363354


When finished, you can close the environment.

In [6]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cpu


In [8]:
REPLAY_BUFFER_SIZE = 10000
VISUALIZE_EVERY = 50 #how many episodes to train before watching unity env simulation
PRINT_EVERY = 10 #how many steps to train before printing training info
DESIRED_EPISODES_AVERAGE = 100
DESIRED_AVERAGE_SCORE = 0.5
TRAIN_EVERY = 4
BATCH_SIZE =64
UPDATE_TARGET_NETWORK_EVERY = TRAIN_EVERY
TAU = 0.001

In [9]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples for experience replay"""
    
    def __init__(self,action_size,buffer_size):
        """Initialize a ReplayBuffer objectParams
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
        """
        
        self.action_size = action_size
        self.buffer_size = buffer_size
        self.memory = deque(maxlen=buffer_size)
        self.experience = namedtuple("Experience",field_names=["state", "action", "reward", "next_state", "done"])
        
    def add(self,state,action,reward,next_state,done):
        "Add a new experience tuple to replay buffer"
        new_experience = self.experience(state,action,reward,next_state,done)
        self.memory.append(new_experience)
        
    def sample(self,batch_size):
        "Get a sample of the buffer of size=batch size"
        experience_batch = random.sample(self.memory,k=batch_size)
        
        states =  torch.from_numpy(np.vstack([e.state  for e in experience_batch if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experience_batch if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experience_batch if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experience_batch if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experience_batch if e is not None]).astype(np.uint8)).to(device)
        
        return (states,actions,rewards,next_states,dones)
    
    def __len__(self):
        """Return the current size of internal memory"""
        return len(self.memory)

In [10]:
class OUNoise:
    """
        Orstein-Ulenbeck process
        used to add noise to actor selections for adding exploratoion
    """
    def __init__(self,size,mu=0.0,theta=0.15,sigma=0.2):
        self.size = size
        self.mu = mu*np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.reset()
        
    def reset(self):
        self.state = copy.copy(self.mu)
        
    def sample(self):
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.random.standard_normal(self.size)
        self.state = x + dx
        
        return self.state

In [11]:
def hidden_init(layer):
    """Used to set the initial weights of hidden layers"""
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

In [12]:

# one actor will be shared by the 2 players, trained by self-play
class Actor(torch.nn.Module):
    def __init__(self,state_size,action_size):
        super(Actor,self).__init__()
        self.action_size = action_size
        self.fc1 = torch.nn.Linear(state_size,400)
        self.fc2 = torch.nn.Linear(400,300)
        self.fc_output = torch.nn.Linear(300,action_size)
        self.reset_parameters()
        
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        y = F.tanh(self.fc_output(x))
        
        return y
    
    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc_output.weight.data.uniform_(-3e-3, 3e-3)

In [13]:
# one critic will be shared by the 2 players, trianed by self-play
class Critic(torch.nn.Module):
    def __init__(self,state_size,action_size):
        super(Critic,self).__init__()
        self.fc1 = torch.nn.Linear(state_size+ action_size,400) #use actions of both agent
        self.fc2 = torch.nn.Linear(400,300)
        self.fc_output = torch.nn.Linear(300,1)
        self.reset_parameters()
        
    def forward(self,x,actions):
        x = torch.cat((x,actions),dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        y = self.fc_output(x)
        
        return y
    
    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc_output.weight.data.uniform_(-3e-4, 3e-4)

In [14]:
class Agent:
    def __init__(self,state_size,action_size,actor_local,replay_buffer,device):
        self.action_size  = action_size
        self.actor_local = actor_local
        self.device = device
        
        self.replay_buffer = replay_buffer
        
        self.noise = OUNoise(size=action_size)
        
    def act(self,state):
        
        state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
        self.actor_local.eval()
        with torch.no_grad():
            action = self.actor_local(state).cpu().data.numpy()
            
        self.actor_local.train()
        noise = self.noise.sample()
        action = np.clip(action + noise ,-1,1)
        
        return action

In [15]:
def update_target_network(local_network,target_network,tau):
    for target_param,local_param in zip(target_network.parameters(),local_network.parameters()):
        target_param.data.copy_(tau*local_param.data + (1.0 - tau)*target_param.data)

In [16]:
replay_buffer = ReplayBuffer(action_size,REPLAY_BUFFER_SIZE)

actor_local = Actor(state_size,action_size).to(device)
critic_local = Critic(state_size,action_size).to(device)

actor_target = Actor(state_size,action_size).to(device)
critic_target = Critic(state_size,action_size).to(device)

update_target_network(actor_local,actor_target,1.0)
update_target_network(critic_local,critic_target,1.0)

critic_optimizer = optim.Adam(critic_local.parameters(),lr=3e-5)
actor_optimizer = optim.Adam(actor_local.parameters(),lr=3e-5)

agent1 = Agent(state_size,action_size,actor_local,replay_buffer,device)
agent2 = Agent(state_size,action_size,actor_local,replay_buffer,device)

In [17]:
def train(batch_size=32,step_num=0,print_every = 100,critic_losses=[],actor_losses = []):
    (states, actions, rewards, next_states, dones) = replay_buffer.sample(batch_size)
    
    actor_target.eval()
    critic_target.eval()
    
    with torch.no_grad():
        next_target_actions = actor_target(next_states)
        target_q_values = critic_target(next_states,next_target_actions)
        target_return = rewards + 0.99*target_q_values
    
    #train critic
    critic_local.train()
    
    current_critic_estimate = critic_local(states,actions)
    critic_mse_loss = F.mse_loss(current_critic_estimate,target_return)
    
    critic_optimizer.zero_grad()
    critic_mse_loss.backward()
    torch.nn.utils.clip_grad_norm_(critic_local.parameters(),1)
    critic_optimizer.step()
    
    critic_losses.append(critic_mse_loss)
    
    #train actor
    local_actions = actor_local(states)
    actor_loss = -critic_local(states,local_actions).mean()
    
    actor_optimizer.zero_grad()
    actor_loss.backward()
    actor_optimizer.step()
    
    actor_losses.append(actor_loss)

In [ ]:
def main(agent1,agent2,episodes,visualize_every,print_every):
    total_rewards = [[],[]]
    critic_losses = []
    actor_losses = []
    episode_single_scores = [] # the max between scores of both agents
    episode_window_scores = deque(maxlen = DESIRED_EPISODES_AVERAGE)
    
    for episode in range(1,episodes +1):
        finished = False
        step_count = 0
        episode_scores = np.zeros(num_agents)
        
        agent1.noise.reset()
        agent2.noise.reset()
        train_mode = not(episode%visualize_every == 0)
        env_info = env.reset(train_mode = train_mode)[brain_name]
        states = env_info.vector_observations
        
        while not finished:
            actions_ag1 = agent1.act(states[0])
            actions_ag2 = agent2.act(states[1])
            env_info = env.step(np.concatenate((actions_ag1,actions_ag2),axis=0))[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done
            
            #add experience to shared replay buffer
            #if np.sum(rewards) != 0.0:
            replay_buffer.add(states[0],actions_ag1,rewards[0],next_states[0],dones[0])
            replay_buffer.add(states[1],actions_ag2,rewards[1],next_states[1],dones[1])
            
            states = next_states
            
            episode_scores += rewards
            finished = dones[0] or dones[1]
            
            if step_count % TRAIN_EVERY == 0 and len(replay_buffer) >= BATCH_SIZE:
                train(critic_losses = critic_losses,actor_losses = actor_losses)
                
            if step_count % UPDATE_TARGET_NETWORK_EVERY == 0 and len(replay_buffer) >= BATCH_SIZE:
                update_target_network(actor_local,actor_target,TAU)
                update_target_network(critic_local,critic_target,TAU)
                
            step_count += 1
            
            
        episode_single_score = max(episode_scores)
        episode_window_scores.append(episode_single_score)
        episode_single_scores.append(episode_single_score)
        
        total_rewards[0].append(episode_scores[0])
        total_rewards[1].append(episode_scores[1])
        
        if episode % print_every == 0:
            print("Episode:{} steps:{} episode_reward:{} last n mean rew.{},individual episode rew.{}".format(episode, step_count,
                                                                                    episode_single_score,np.mean(episode_window_scores),episode_scores))
                
        
        if np.mean(episode_window_scores) >= DESIRED_AVERAGE_SCORE:
            print("Solved in {} episodes".format(episode))
            checkpoint_name = "checkpoint_solved_"+str(episode)+".pth"
            break
            
    return total_rewards,critic_losses,actor_losses,episode_single_scores

train_rewards,critic_losses,actor_losses,episode_single_scores = main(agent1,agent2,1000,VISUALIZE_EVERY,PRINT_EVERY)

Episode:10 steps:30 episode_reward:0.10000000149011612 last n mean rew.0.010000000149011612,individual episode rew.[-0.01  0.1 ]
Episode:20 steps:21 episode_reward:0.0 last n mean rew.0.005000000074505806,individual episode rew.[-0.01  0.  ]
Episode:30 steps:15 episode_reward:0.0 last n mean rew.0.0033333333830038708,individual episode rew.[-0.01  0.  ]
Episode:40 steps:15 episode_reward:0.0 last n mean rew.0.002500000037252903,individual episode rew.[ 0.   -0.01]
Episode:50 steps:15 episode_reward:0.0 last n mean rew.0.0020000000298023225,individual episode rew.[ 0.   -0.01]
Episode:60 steps:15 episode_reward:0.0 last n mean rew.0.0016666666915019354,individual episode rew.[-0.01  0.  ]
Episode:70 steps:15 episode_reward:0.0 last n mean rew.0.0014285714498588017,individual episode rew.[-0.01  0.  ]
Episode:80 steps:15 episode_reward:0.0 last n mean rew.0.0012500000186264515,individual episode rew.[-0.01  0.  ]
Episode:90 steps:15 episode_reward:0.0 last n mean rew.0.001111111127667956

In [ ]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    agent1.noise.reset()
    agent2.noise.reset()
    while True:
        actions_ag1 = agent1.act(states[0])
        actions_ag2 = agent2.act(states[1])
        print(actions_ag1,actions_ag2)
        env_info = env.step(np.concatenate((actions_ag1,actions_ag2),axis=0))[brain_name]
        next_states = env_info.vector_observations        # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

#### Agent 1 rewards

In [ ]:
plt.plot(train_rewards[0])
plt.show()

#### Agent 2 rewards

In [ ]:
plt.plot(train_rewards[1])
plt.show()

### episode rewards

In [ ]:
plt.plot(episode_single_scores)
plt.show()

#### Critic losses

In [ ]:
plt.plot(critic_losses)
plt.show()

### Actor losses

In [ ]:
plt.plot(actor_losses)
plt.show()

In [ ]:
replay_buffer.sample(100)[2]